In [ ]:
import pyspark.sql.functions as F
from data_location import DELTA_LOCATION

from spark_bi.spark import FutPathlingContext

pc = FutPathlingContext.create(app_name="example-spark-app")
delta_lake = pc.read.delta(DELTA_LOCATION)

25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function display replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function member_of replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function subsumes replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function designation replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function property_string replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function property_code replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function property_integer replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function property_boolean replaced a previously registered function.
25/12/12 10:40:25 WARN SimpleFunctionRegistry: The function property_deci

In [ ]:
careteams = delta_lake.view(
    resource="CareTeam",
    select=[
        {
            "column": [
                {"name": "careteam_id", "path": "getResourceKey()"},
                {"name": "org_id", "path": "managingOrganization.first().getReferenceKey()"},
            ]
        }
    ],
)
careteams.filter(F.col("org_id").isNotNull()).head(5)

[Row(careteam_id='CareTeam/3000148060', org_id='Organization/3000038806'),
 Row(careteam_id='CareTeam/3000148061', org_id='Organization/3000029719')]

In [46]:
organizations = delta_lake.view(
    resource="Organization",
    select=[
        {
            "column": [
                {"name": "org_id", "path": "getResourceKey()"},
                {
                    "name": "municipality_code",
                    "path": "extension('http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-organization-municipalityCode').valueString",
                },
            ]
        }
    ],
)
organizations.filter(F.col("municipality_code").isNotNull()).head(5)

25/12/12 10:46:47 WARN Collection: Traversing a choice element `valueString` without using ofType() is not portable and may not work in some FHIRPath implementations. Consider using ofType() to specify the type of element you want to traverse.


[Row(org_id='Organization/3000000064', municipality_code='0787'),
 Row(org_id='Organization/3000000069', municipality_code='0265'),
 Row(org_id='Organization/3000000072', municipality_code='0173'),
 Row(org_id='Organization/3000000088', municipality_code='0360'),
 Row(org_id='Organization/3000000090', municipality_code='0787')]

In [47]:
joined = careteams.join(organizations, careteams.org_id == organizations.org_id, how="left")
joined.head(5)

[Row(careteam_id='CareTeam/3000000001', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000002', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000005', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000006', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000007', org_id=None, org_id=None, municipality_code=None)]

In [48]:
joined.filter(F.col("municipality_code").isNotNull()).head(5)

[Row(careteam_id='CareTeam/3000148060', org_id='Organization/3000038806', org_id='Organization/3000038806', municipality_code='0851'),
 Row(careteam_id='CareTeam/3000148061', org_id='Organization/3000029719', org_id='Organization/3000029719', municipality_code='0849')]

På TRIFORKs testmiljø er der kun 2 careteams der er tilknyttet en kommune. De kan vises i en oversigt:

In [51]:
joined.groupBy("municipality_code").agg(
    F.countDistinct("careteam_id").alias("n_careteams")
).orderBy("n_careteams", ascending=False).toPandas()

,municipality_code,n_careteams
0,None,31
1,0851,1
2,0849,1
